## New York City Green Taxi Data Cleaning

The notebook contains a sequence of steps to explore green taxi data for year 2020. I used PySpark in Jupyter Notebook so it can show directly the dataframe when calling action operations. The goal of this notebook is to output the cleaned dataset to build a web-based dashboard using Plotly-Dash.

### Import modules

At a very first, import the `findspark` module to get the Spark home in the local machine.

In [1]:
import findspark

findspark.find()
findspark.init()

Then, I can import all functionalities of Spark.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    to_timestamp, 
    date_format,
    year, 
    month, 
    hour, 
    col, 
    round,
    ceil,
    floor,
    minute, 
    second,
    when
)

In [3]:
import plotly.express as px

### Initialize SparkSession

SparkSession is created using local thread of four and can be seen in `localhost:4050` with "NYC Green Taxi" as the application name.

In [4]:
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("NYC Green Taxi") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

22/04/20 14:48:08 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.43.60 instead (on interface wlp0s20f3)
22/04/20 14:48:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/20 14:48:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Set this configuration to print the dataframe when calling `.show` operation.

In [5]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

### Read the data

The dataset that will be used is NYC green taxi data for all months in 2020. Let us see the columns.

In [6]:
df = spark.read \
    .format('csv') \
    .option("inferSchema", "true") \
    .option("header","true") \
    .load("/home/thomas/data/nyc_taxi/green_taxi/*2020*.csv")
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



Write the dataframe into Parquet format. I did this to make a memory-efficient application when I re-read the data using the chosen format.

In [7]:
df \
    .write \
    .mode('overwrite') \
    .parquet('staging/nyc-green-2020')

Re-read the data with the same variable as before, `df`, but for now using the Parquet file I have wrote earlier.

In [8]:
df = spark.read \
    .format('parquet') \
    .option("header","true") \
    .load("staging/nyc-green-2020")
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



The data dictionary of this dataset is available [here](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_green.pdf). There are some codes identifying the columns, such as:

1. `VendorID` contains a code indicating the LPEP provider
    * `1` : Creative Mobile Technologies, LLC
    * `2` : VeriFone Inc.


2. `RateCodeID` is the final rate code in effect at the end of the trip.
    * `1` : Standard rate
    * `2` : JFK
    * `3` : Newark
    * `4` : Nassau or Westchester
    * `5` : Negotiated fare
    * `6` : Group ride
    
    
3. `payment_type` is a numeric code signifying how the passenger paid for the trip.
    * 1 : Credit card
    * 2 : Cash
    * 3 : No charge
    * 4 : Dispute
    * 5 : Unknown
    * 6 : Voided trip
    

4. `PULocationID` and `DOLocationID` contain a code of pick up and drop off location zones which is referencing to this [file](https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv).


See the size of the dataset

In [9]:
df.count(), len(df.columns)

(1734051, 20)

In [10]:
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|         1|         264|         264|              5|          0.0|        3.5|  0.5|    0.

See the summary of the dataset

In [11]:
df.summary()

summary,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,1205959,1734051,1734051,1205959,1205959,1734051,1734051,1205959,1734051,1734051,1734051,1734051,1734051,1734051,0,1734051,1734051,1205959,1205954,1205959
mean,1.8298316941123205,null,null,null,1.118096054675159,108.24825682750969,128.7862756055041,1.2913266537253754,17.292239957186986,16.984788048339592,0.5418312033498438,0.3480491346563625,1.263317930095531,0.4129914979431395,null,0.29325262059946694,20.16313390433051,1.4599285713693417,1.0262953644998067,0.4842113206170359
stddev,0.37578074811450785,null,null,null,0.7140994719763627,70.80139561161255,76.42576845637467,0.9547199115132876,1198.1647047946237,13.805149119347435,0.9482827432384792,0.23257857098804421,2.2282006210319514,1.6595631387406184,null,0.04914765352186226,15.273077679603345,0.5219627145750482,0.16001231057922746,1.047321940025361
min,1,2008-12-31 22:06:48,2008-12-31 23:12:08,N,1,1,1,0,-33.69,-210.0,-4.5,-0.5,-10.56,-6.12,null,-0.3,-210.3,1,1,-2.75
25%,2,null,null,null,1,52,63,1,1.13,7.5,0.0,0.0,0.0,0.0,null,0.3,9.36,1,1,0.0
50%,2,null,null,null,1,81,127,1,2.21,12.5,0.0,0.5,0.0,0.0,null,0.3,15.7,1,1,0.0
75%,2,null,null,null,1,166,194,1,4.87,22.11,1.0,0.5,2.75,0.0,null,0.3,26.0,2,1,0.0
max,2,2041-08-17 16:24:38,2041-08-17 16:27:20,Y,99,265,265,9,205654.12,803.0,16.74,3.55,641.2,96.12,null,0.3,803.8,5,2,2.75


In this summary, I find something interesting. 
* The vendor is dominating by `VendorID` = 2, that is Verifone Inc. Also, it has many null values.
* Both pick up and drop off dates contain typos as this is just a dataset for 2020.
* The `RateCodeID` contains a number outside the listed categories.
* The number of passenger exceeds 4 for green taxies, based on this [information](https://freetoursbyfoot.com/how-to-get-a-taxi-in-nyc/). The zero in passenger maybe a cancelled trip or mistyped by the driver.
* The `trip_distance` has unnatural minimum and maximum values.
* The `fare_amount`, `extra`, `mta_tax`,`tip_amount`, `total_amount`,`improvement_surcharge`, and `congestion_surcharge` minimum value has a negative value.
* The `ehail_fee` are null for all records.
* Null values also come in `store_and_fwd_flag`, `RateCodeID`, `passenger_count`, `payment_type`, `trip_type`, and `congestion_surcharge`.

I will investigate further for all questionable values for each column.

### Explore the columns

In this section, I will check the columns that have null values. I use `.filter` and `.isNull()` to filter all records with null values in selected columns.

In [12]:
df1 = df.filter(df.VendorID.isNull())
df1.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|    null| 2020-01-01 00:39:00|  2020-01-01 01:11:00|              null|      null|         143|          45|           null|         5.57|      25.78| 2.75|    0.

Interesting! I see a pattern from this first 20 records of null data in `VendorID` column. The datetime, pick up and drop off location id, and `trip_distance` values are doubled with variation in `fare amount`. Also, the `extra` column is filled with either 0.00 or 2.75. I might delete all these strange records.

Assign the non-null values of `VendorID` column to a new variable.

In [13]:
df_vendor = df.filter(df.VendorID.isNotNull())

Create a new column named `do_year` for drop off year and extract the value from the drop off datetime using `.withColumn` method and `year` function from `pyspark.sql`.

In [14]:
df_year = df_vendor.withColumn('do_year', year('lpep_dropoff_datetime'))

Count the records which has year before 2020 and show some of the data.

In [15]:
df_year.filter(col('do_year') < 2020).count()

43

In [16]:
df_year.filter(col('do_year') < 2020).show(43)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|do_year|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|         1|         264|         264|              5|          0.0|

In [17]:
df_year.filter(col('do_year') > 2020).count()

12

In [18]:
df_year.filter(col('do_year') > 2020).show(12)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|do_year|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+
|       2| 2041-08-17 16:24:38|  2041-08-17 16:27:20|                 N|         1|         193|         193|              1|          0.0|

Many of the records are the orders that picked up around midnight in the new year's eve. I will delete the records since I only focused on the orders that finished in 2020 and keep them if picked up in the last day of December and finished in the first of January 2020.

In [19]:
df_2020 = df_year.filter(col('do_year') == 2020)

In [20]:
df_2020.withColumn('pu_year', year('lpep_pickup_datetime')).filter(col('pu_year') == 2019).show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|do_year|pu_year|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------+
|       2| 2019-12-31 23:59:39|  2020-01-01 00:06:24|                 N|         1|         179|         179|      

I see a possibly bad record with a trip duration almost 24 hours for pick up datetime at 2019-12-31 20:27:53 and drop off datetime at 2020-01-01 19:45:52 while the trip distance is only 10.19 miles with total fare is only \$36.96. Did the cab goes extremely slow? It does not make sense to order with this kind of trip.

From the `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns, I convert the data type into timestamp, assign new names for both columns. Also, I decide to create two new columns to store trip durations in two units, minute and second.

In [21]:
df_convert_date = df_2020 \
    .withColumn('pu_datetime', to_timestamp('lpep_pickup_datetime')) \
    .withColumn('do_datetime', to_timestamp('lpep_dropoff_datetime')) \
    .drop('lpep_pickup_datetime','lpep_dropoff_datetime')

    .withColumn('pu_day', date_format('pu_datetime', 'EEE')) \
    .withColumn('pu_hour', hour('pu_datetime')) \
    .withColumn('pu_month', month('pu_datetime')) \
    .withColumn('trip_duration_second', col('do_datetime').cast('long') - col('pu_datetime').cast('long')) \
    .withColumn('trip_duration_minute', round(col('trip_duration_second')/60))

In [22]:
df_convert_date.show()

+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------------------+-------------------+
|VendorID|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|do_year|        pu_datetime|        do_datetime|
+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------------------+-------------------+
|       2|                 N|         5|          66|          65|              2|         1.28|       20.0|  0.0|    0.0|      4.06|         0.0|  

Check both `fare_amount` and `total_amount` that have values below zero.

In [23]:
df_convert_date.filter((col('fare_amount') < 0) & (col('total_amount') < 0)).count()

4195

In [24]:
df_convert_date \
    .filter((col('fare_amount') < 0) & (col('total_amount') < 0)) \
    .show()

+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------------------+-------------------+
|VendorID|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|do_year|        pu_datetime|        do_datetime|
+--------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-------+-------------------+-------------------+
|       2|                 N|         1|          80|         256|              2|         0.71|       -5.5| -0.5|   -0.5|       0.0|         0.0|  

With 4195 records, I think the negative sign in all columns about charges to passengers is a typo. So, the value will be multiplied by -1 to make it positive. I am also filter out the records that have `fare_amount` below \$2.50 which is a minimum charge in every order. Other than that, I'm only going to select a few columns.

In [25]:
df_fare = df_convert_date \
    .withColumn('fare_amount', 
                when(col('fare_amount') < 0, col('fare_amount') * -1).otherwise(col('fare_amount'))) \
    .withColumn('total_amount', 
                when(col('total_amount') < 0, col('total_amount') * -1).otherwise(col('total_amount'))) \
    .filter(col('fare_amount') >= 2.5) \
    .select(
        'VendorID',
        'pu_datetime',
        'do_datetime',
        'do_year',
        'PULocationID',
        'DOLocationID',
        'passenger_count',
        'trip_distance',
        'trip_type',
        'payment_type',
        'fare_amount',
        'total_amount'
    )

Now, let us check `trip_distance` column. From the summary, it also has zero or negative values.

In [26]:
df_fare.filter(col('trip_distance') <= 0).count()

57705

In [27]:
df_fare.filter(col('trip_distance') <= 0).show()

+--------+-------------------+-------------------+-------+------------+------------+---------------+-------------+---------+------------+-----------+------------+
|VendorID|        pu_datetime|        do_datetime|do_year|PULocationID|DOLocationID|passenger_count|trip_distance|trip_type|payment_type|fare_amount|total_amount|
+--------+-------------------+-------------------+-------+------------+------------+---------------+-------------+---------+------------+-----------+------------+
|       1|2020-01-01 00:23:42|2020-01-01 00:48:02|   2020|          92|          51|              1|          0.0|        1|           1|       26.2|       33.12|
|       2|2020-01-01 00:52:51|2020-01-01 00:54:17|   2020|          36|          36|              1|          0.0|        2|           1|       50.0|        70.3|
|       1|2020-01-01 00:43:43|2020-01-01 01:14:39|   2020|          37|         150|              1|          0.0|        1|           1|       28.2|        29.0|
|       2|2020-01-01 0

Surprisingly, the number of bad records of this kind is very high, greater than 50,000 records. This could happen possibly by an error originating from the meter. For now, I will exclude them from the valid data.

In [28]:
df_distance = df_fare.filter(col('trip_distance') > 0)

I filtered the records based on `trip_distance` and `fare_amount` because I saw strange values between the distance and the amount passenger paid. The assumption here was by using a standard metered fare, that is, \\$2.50 for initial charge and \\$0.50 every 1/5 mile. The filter is done with the trip distance larger than 40 miles, though the less distance trips have this behaviour too.

In [29]:
## Check trip distance again
filters = df_distance \
        .filter(
            (col('fare_amount') > 0) &
            (col('trip_distance') > 40) &
            (col('fare_amount') < 2.5 + (floor(col('trip_distance')/0.2)*0.5))  # standard metering assumption
        ) \
        .select(
            'trip_distance',
            'fare_amount',
            'total_amount'
        )

filters.count()

36

In [30]:
df_dist_fare = df_distance.join(filters,['trip_distance','fare_amount','total_amount'],'leftanti')

In [31]:
df_dist_fare.filter('passenger_count == 0 or passenger_count > 4').count()

53082

The cab can only afford a maximum of four passengers. However, the summary tells that there are records with passengers larger than four. I delete all records with the number of passengers beyond 1 and 4, many rows to delete.

In [32]:
df_passenger = df_dist_fare.filter('passenger_count > 0 and passenger_count <= 4')

In [33]:
df_passenger.summary()

summary,trip_distance,fare_amount,total_amount,VendorID,do_year,PULocationID,DOLocationID,passenger_count,trip_type,payment_type
count,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261
mean,2.9816072481727,12.566210879780193,15.667480594051057,1.8527618616092845,2020.0,100.53531952440746,129.07023731014868,1.11610981223762,1.0182240766201855,1.4668992103725622
stddev,3.674697238235831,11.275975839805042,12.967158377390165,0.35434331397429614,1.593400037178569...,66.68745190250091,76.59338204358056,0.400618707370957,0.13376089137806374,0.5192059946225511
min,0.01,2.5,2.5,1,2020,1,1,1,1,1
25%,1.05,6.5,8.3,2,2020,49,65,1,1,1
50%,1.79,9.0,11.76,2,2020,75,129,1,1,1
75%,3.35,14.5,18.27,2,2020,130,193,1,1,2
max,171.01,803.0,803.8,2,2020,265,265,4,2,5


After finishing check the columns, I will create new dimensions that can be extracted from the timestamp columns. I will parse each to year, month, date, hour, and minute, as well as trip duration in minute and second for the unit time.

In [34]:
df_passenger.columns

['trip_distance',
 'fare_amount',
 'total_amount',
 'VendorID',
 'pu_datetime',
 'do_datetime',
 'do_year',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_type',
 'payment_type']

In [35]:
df_features = df_passenger \
        .withColumn('pu_year', year('pu_datetime')) \
        .withColumn('pu_month', month('pu_datetime')) \
        .withColumn('pu_date', date_format('pu_datetime','yyyy-MM-dd')) \
        .withColumn('pu_day', date_format('pu_datetime','EEE')) \
        .withColumn('pu_hour', hour('pu_datetime')) \
        .withColumn('pu_minute', minute('pu_datetime')) \
        .withColumn('do_month', month('do_datetime')) \
        .withColumn('do_date', date_format('do_datetime','yyyy-MM-dd')) \
        .withColumn('do_day', date_format('do_datetime','EEE')) \
        .withColumn('do_hour', hour('do_datetime')) \
        .withColumn('do_minute', minute('do_datetime')) \
        .withColumn('trip_dur_second', col('do_datetime').cast('long') - col('pu_datetime').cast('long')) \
        .withColumn('trip_dur_minute', round(col('trip_dur_second') / 60))        

df_features.summary()

summary,trip_distance,fare_amount,total_amount,VendorID,do_year,PULocationID,DOLocationID,passenger_count,trip_type,payment_type,pu_year,pu_month,pu_date,pu_day,pu_hour,pu_minute,do_month,do_date,do_day,do_hour,do_minute,trip_dur_second,trip_dur_minute
count,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261,1090261
mean,2.9816072481727,12.566210879780193,15.667480594051057,1.8527618616092845,2020.0,100.53531952440746,129.07023731014868,1.11610981223762,1.0182240766201855,1.4668992103725622,2019.9999926623075,3.8737935228353577,null,null,13.968082871899481,29.612248810147296,3.874086113325158,null,null,14.052873577978117,29.67762123014581,1140.532201922292,19.01441031092555
stddev,3.674697238235831,11.275975839805042,12.967158377390165,0.35434331397429614,1.593400037178569...,66.68745190250091,76.59338204358056,0.400618707370957,0.13376089137806374,0.5192059946225511,0.002708808856762296,3.4176537566376592,null,null,5.532690552313007,17.290527819263094,3.41755679496102,null,null,5.601914554241017,17.43853324891742,5274.262528024155,87.90507197567484
min,0.01,2.5,2.5,1,2020,1,1,1,1,1,2019,1,2019-12-31,Fri,0,0,1,2020-01-01,Fri,0,0,-326180,-5436.0
25%,1.05,6.5,8.3,2,2020,49,65,1,1,1,2020,1,null,null,10,15,1,null,null,10,14,375,6.0
50%,1.79,9.0,11.76,2,2020,75,129,1,1,1,2020,2,null,null,15,30,2,null,null,15,30,616,10.0
75%,3.35,14.5,18.27,2,2020,130,193,1,1,2,2020,6,null,null,18,45,6,null,null,18,45,989,16.0
max,171.01,803.0,803.8,2,2020,265,265,4,2,5,2020,12,2020-12-31,Wed,23,59,12,2020-12-31,Wed,23,59,86398,1440.0


In [36]:
df_features.columns

['trip_distance',
 'fare_amount',
 'total_amount',
 'VendorID',
 'pu_datetime',
 'do_datetime',
 'do_year',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_type',
 'payment_type',
 'pu_year',
 'pu_month',
 'pu_date',
 'pu_day',
 'pu_hour',
 'pu_minute',
 'do_month',
 'do_date',
 'do_day',
 'do_hour',
 'do_minute',
 'trip_dur_second',
 'trip_dur_minute']

From the summary of latest dataframe, the trip duration has records with negative value. The negative values maybe caused by a malfunction of the meter while engaged and/or disengaged. So the records will be deleted. 

On the other hand, the maximum value of trip duration is 1440 minutes, which is 24 hours, whereas 75% of the data only have trips not exceeds than 16 minutes. I must check the validity of the time written in the dataset.

In [37]:
df_features.filter(col('trip_dur_minute') < 1).count()

7975

In [38]:
df_pos_dur = df_features \
        .filter(~(col('trip_dur_minute') < 1)) \
        .select(
             'VendorID',
             'PULocationID',
             'DOLocationID',
             'pu_datetime',
             'do_datetime',
             'pu_year',
             'pu_month',
             'pu_date',
             'pu_day',
             'pu_hour',
             'pu_minute',
             'do_year',
             'do_month',
             'do_date',
             'do_day',
             'do_hour',
             'do_minute',
             'trip_dur_second',
             'trip_dur_minute',             
             'trip_distance',
             'fare_amount',
             'total_amount',
             'passenger_count',
             'trip_type',
             'payment_type'
            )
df_pos_dur.summary()

summary,VendorID,PULocationID,DOLocationID,pu_year,pu_month,pu_date,pu_day,pu_hour,pu_minute,do_year,do_month,do_date,do_day,do_hour,do_minute,trip_dur_second,trip_dur_minute,trip_distance,fare_amount,total_amount,passenger_count,trip_type,payment_type
count,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286,1082286
mean,1.8528780747417966,100.441333436818,129.10301343637448,2019.9999926082385,3.8687287833345345,null,null,13.976065476223475,29.61359843885997,2020.0,3.8690235298248337,null,null,14.06147820446721,29.679071890424527,1149.1466895072097,19.159543780479467,2.9999745631006838,12.578058729393186,15.689708995544233,1.1160950063107165,1.015855328443683,1.4643079555681215
stddev,0.3542275826410142,66.61900037685015,76.55016683253346,0.002718770623568...,3.415980340062286,null,null,5.52726388793228,17.2904275052102,0.0,3.4158830529749156,null,null,5.596936431037464,17.439465930682555,5283.406144820507,88.05715176771561,3.6794724784205717,11.220433199885653,12.919119578563222,0.40062238159818936,0.12491577731100272,0.5164384108622434
min,1,1,1,2019,1,2019-12-31,Fri,0,0,2020,1,2020-01-01,Fri,0,0,30,1.0,0.01,2.5,2.5,1,1,1
25%,2,49,65,2020,1,null,null,10,15,2020,1,null,null,10,14,380,6.0,1.06,6.5,8.3,1,1,1
50%,2,75,129,2020,2,null,null,15,30,2020,2,null,null,15,30,620,10.0,1.8,9.0,11.8,1,1,1
75%,2,130,193,2020,6,null,null,18,45,2020,6,null,null,18,45,993,17.0,3.37,14.5,18.3,1,1,2
max,2,265,265,2020,12,2020-12-31,Wed,23,59,2020,12,2020-12-31,Wed,23,59,86398,1440.0,171.01,803.0,803.8,4,2,5


For now, I will give a flag if the record is doubtful by giving a boundary for calculated average velocity. The lower boundary is 3 and the upper is 100. Any records beyond it will be flagged as doubtful.

In [39]:
df_vel = df_pos_dur \
        .withColumn('avg_velocity',(col('trip_distance') / (col('trip_dur_minute') / 60))) \
        .withColumn('doubtful', when((col('avg_velocity') < 3) | (col('avg_velocity') >100) , 1).otherwise(0))

In [40]:
df_vel.filter(col('avg_velocity') > 100).show()

22/04/20 14:50:09 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+------------+------------+-------------------+-------------------+-------+--------+----------+------+-------+---------+-------+--------+----------+------+-------+---------+---------------+---------------+-------------+-----------+------------+---------------+---------+------------+------------------+--------+
|VendorID|PULocationID|DOLocationID|        pu_datetime|        do_datetime|pu_year|pu_month|   pu_date|pu_day|pu_hour|pu_minute|do_year|do_month|   do_date|do_day|do_hour|do_minute|trip_dur_second|trip_dur_minute|trip_distance|fare_amount|total_amount|passenger_count|trip_type|payment_type|      avg_velocity|doubtful|
+--------+------------+------------+-------------------+-------------------+-------+--------+----------+------+-------+---------+-------+--------+----------+------+-------+---------+---------------+---------------+-------------+-----------+------------+---------------+---------+------------+------------------+--------+
|       1|         177|         177|2

### Join dataframe

This section will focus on joining other dataframes with the dataframe I am currently working on. I would replace the codes for certain fields, such as `vendorID`, `payment_type`, and `trip_type`, with their actual values by concatenating them with the dataframe that contains information about the code.

Let us create the dataframes by setting up the code, actual values, and column names.

In [41]:
vendor = [
    (1, 'Creative Mobile Technologies, LLC'),
    (2, 'VeriFone Inc')
]

vendor_cols = ['id','vendor']

In [42]:
payment = [
    (1, 'Credit card'),
    (2, 'Cash'),
    (3, 'No charge'),
    (4, 'Dispute'),
    (5, 'Unknown'),
    (6, 'Voided trip')
]

payment_cols = ['id','payment']

In [43]:
trip = [
    (1, 'Street-hail'),
    (2, 'Dispatch')
]

trip_cols = ['id','trip']

In [44]:
df_vendor = spark.createDataFrame(vendor, vendor_cols)
df_payment = spark.createDataFrame(payment, payment_cols)
df_trip = spark.createDataFrame(trip, trip_cols)

In [45]:
df_join = df_vel \
        .join(df_vendor, df_vel.VendorID == df_vendor.id, 'inner').drop(*('VendorID','id')) \
        .join(df_payment, df_vel.payment_type == df_payment.id, 'inner').drop(*('payment_type','id')) \
        .join(df_trip, df_vel.trip_type == df_trip.id, 'inner').drop(*('trip_type','id')) \
        .select(
             'vendor',
             'PULocationID',
             'DOLocationID',
             'pu_year',
             'pu_month',
             'pu_date',
             'pu_day',
             'pu_hour',
             'pu_minute',
             'do_year',
             'do_month',
             'do_date',
             'do_day',
             'do_hour',
             'do_minute',
             'trip_dur_minute',             
             'trip_distance',
             'fare_amount',
             'total_amount',
             'passenger_count',
             'trip',
             'payment'
        )

In [46]:
df_join.count(), len(df_join.columns)

(1082286, 22)

### Save the data

The dataset is saved into Parquet format.

In [47]:
df_join \
    .write \
    .mode('overwrite') \
    .parquet('output/nyc-green-2020')